In [ ]:
import os
import pandas as pd
import unicodedata
from tqdm import tqdm
import pycountry
database = "data/raw"

In [ ]:
def normalize(col):
    s = str(col)
    s2 = unicodedata.normalize("NFD", s).encode("ascii","ignore").decode().upper().strip()
    return s2

# Portuguese to English translation mapping 
translation = {
 "EMPRESA (SIGLA)".upper(): "airline_abbrev",
 "EMPRESA (NOME)".upper(): "airline_name",
 "EMPRESA (NACIONALIDADE)".upper(): "airline_nationality",
 "ANO": "year",
 "MES": "month",
 "AEROPORTO DE ORIGEM (SIGLA)".upper(): "origin_code",
 "AEROPORTO DE ORIGEM (NOME)".upper(): "origin_airport",
 "AEROPORTO DE ORIGEM (UF)".upper(): "origin_state",
 "AEROPORTO DE ORIGEM (REGIAO)".upper(): "origin_region",
 "AEROPORTO DE ORIGEM (PAIS)".upper(): "origin_country",
 "AEROPORTO DE ORIGEM (CONTINENTE)".upper(): "origin_continent",
 "AEROPORTO DE DESTINO (SIGLA)".upper(): "destination_code",
 "AEROPORTO DE DESTINO (NOME)".upper(): "destination_airport",
 "AEROPORTO DE DESTINO (UF)".upper(): "destination_state",
 "AEROPORTO DE DESTINO (REGIAO)".upper(): "destination_region",
 "AEROPORTO DE DESTINO (PAIS)".upper(): "destination_country",
 "AEROPORTO DE DESTINO (CONTINENTE)".upper(): "destination_continent",
 "NATUREZA": "flight_type",
 "GRUPO DE VOO": "flight_group",
 "PASSAGEIROS PAGOS": "passengers_paying",
 "PASSAGEIROS GRATIS": "passengers_nonpaying",
 "CARGA PAGA (KG)".upper(): "cargo_paid_kg",
 "CARGA GRATIS (KG)".upper(): "cargo_free_kg",
 "CORREIO (KG)".upper(): "mail_kg",
 "ASK": "available_seat_km",
 "RPK": "revenue_passenger_km",
 "ATK": "available_tonne_km",
 "RTK": "revenue_tonne_km",
 "COMBUSTIVEL (LITROS)".upper(): "fuel_liters",
 "DISTANCIA VOADA (KM)".upper(): "distance_km",
 "DECOLAGENS": "takeoffs",
 "CARGA PAGA KM": "cargo_paid_km",
 "CARGA GRATIS KM": "cargo_free_km",
 "CORREIO KM": "mail_km",
 "ASSENTOS": "seats",
 "PAYLOAD": "payload",
 "HORAS VOADAS": "hours_flown",
 "BAGAGEM (KG)".upper(): "baggage_kg"
}

region_translation = {
    "NORTE": "North",
    "NORDESTE": "Northeast",
    "CENTRO-OESTE": "Central-West",
    "SUDESTE": "Southeast",
    "SUL": "South"
}



In [ ]:
def country_to_english(name):
    if pd.isna(name) or name.upper() == "UNKNOWN":
        return "UNKNOWN"
    try:
        # Some names may need manual corrections if not recognized
        return pycountry.countries.lookup(name).name
    except LookupError:
        return name  # fallback: leave original if not found

In [ ]:
frames = []
for fname in tqdm(os.listdir(database)):
    if not fname.lower().endswith(".csv"):
        continue
    path = os.path.join(database, fname)
    try:
        # try semicolon delimiter first (ANAC often uses ;)
        df = pd.read_csv(path, sep=";", encoding="latin1", low_memory=False)
    except Exception:
        df = pd.read_csv(path, sep=",", encoding="latin1", low_memory=False)
    
    # normalize column names
    df.columns = [normalize(c) for c in df.columns]
    # translate
    rename_map = {col: translation[col] for col in df.columns if col in translation}
    df = df.rename(columns=rename_map)
    
    frames.append(df)
    print(f"Loaded {fname} with {len(df)} rows")

flights = pd.concat(frames, ignore_index=True)
print("Combined rows:", len(flights))

In [ ]:
flights["passengers_total"] = flights["passengers_paying"].fillna(0) + flights.get("passengers_nonpaying", 0).fillna(0)
flights = flights[flights["passengers_total"] > 0]
print("Rows with passenger flights only:", len(flights))

In [ ]:
flights["origin_country"] = flights["origin_country"].fillna("UNKNOWN")
flights["origin_region"] = flights["origin_region"].fillna("UNKNOWN")
flights["destination_region"] = flights["destination_region"].fillna("UNKNOWN")

# Translate Brazilian regions to English
flights["origin_region"] = flights["origin_region"].map(region_translation).fillna(flights["origin_region"])
flights["destination_region"] = flights["destination_region"].map(region_translation).fillna(flights["destination_region"])

flights["year"] = flights["year"].astype(int)
flights["month"] = flights["month"].astype(int)
flights["year_month"] = flights["year"].astype(str) + "-" + flights["month"].astype(str).str.zfill(2)
flights["flight_type_norm"] = flights["flight_type"].astype(str).str.lower()

In [ ]:
flights

In [ ]:
intl_rows = flights[flights["flight_type_norm"] == "internacional"]
intl_rows

In [ ]:
intl_mask = flights["flight_type_norm"].str.contains("intern", na=False)

intl_summary = (flights.loc[intl_mask]
    .groupby(["year_month", "destination_region", "origin_country"], dropna=False, as_index=False)
    .agg(arriving_passengers=("passengers_paying", "sum")))

intl_summary["arriving_passengers"] = intl_summary["arriving_passengers"].astype(int)
intl_summary.to_csv("intl_arrivals_by_region_country.csv", index=False)
intl_summary.head(50)

In [ ]:
print(flights["destination_region"].unique())

# Optional: also see counts per region
print(flights["destination_region"].value_counts())

In [ ]:
print(flights["origin_country"].unique())

# Optional: also see counts per region
print(flights["origin_country"].value_counts())

In [ ]:
# Cell 8: Domestic arrivals
dom_mask = ~intl_mask

dom_summary = (flights.loc[dom_mask]
    .groupby(["year_month", "destination_region", "origin_region"], dropna=False, as_index=False)
    .agg(arriving_passengers=("passengers_paying", "sum")))

dom_summary["arriving_passengers"] = dom_summary["arriving_passengers"].astype(int)
dom_summary.to_csv("domestic_arrivals_by_region.csv", index=False)
dom_summary.head()

In [ ]:
dom_flights = flights.loc[dom_mask]
dom_flights

In [ ]:
int_flights = flights.loc[intl_mask]
int_flights

In [ ]:
num_unknown = (df["origin_country"] == "UNKNOWN").sum()
print("Number of rows with origin_country = UNKNOWN:", num_unknown)